script to download the data, annotate the training and testing data and save them to another folder to view them

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import json
import cv2
import numpy as np
from PIL import Image
import config
import glioma_utils


In [ ]:
def annotate_and_save_images(input_dir_path, output_dir_path, color_map):
    ''' 
    Aelkhate : This function loops in all the images and annotates it using the information in the json
    
    '''
    if not os.path.exists(output_dir_path):
        os.makedirs(output_dir_path)
    for root, _, files in os.walk(input_dir_path):
        for file in files:
            if file.lower().endswith((".png", ".jpg")):
                base_name = os.path.splitext(file)[0]
                img_path = os.path.join(root, file)
                json_path = os.path.join(root, base_name + ".json")
                
                # Read image using OpenCV
                img = cv2.imread(img_path)
                if img is None:
                    print(f"Error reading {img_path}. Skipping.")
                    continue

                # Create a copy for drawing (we will draw directly on this image)
                annotated_img = img.copy()

                # Load the JSON annotation file
                with open(json_path, 'r') as json_file:
                    annotations = json.load(json_file)
                    for shape in annotations['shapes']:
                        label = shape['label']
                        points = shape['points']
                        color = color_map.get(label, (0, 0, 255))  # Default color: red
                        
                        # Convert points to integer
                        points = np.array(points, dtype=np.int32)
                        
                        # Draw polygon based on points
                        cv2.polylines(annotated_img, [points], isClosed=True, color=color, thickness=2)
                        print(f"Drawn {label} at coordinates: {points[:2]}... ({len(points)} points)")

                # Save the annotated image as PNG
                output_img_path = os.path.join(output_dir_path, base_name + ".png")
                cv2.imwrite(output_img_path, annotated_img)

                print(f"Saved annotated image at {output_img_path}")



In [ ]:
input_dir_path_training  = config.original_data_fixed_train_path
output_dir_path_training = config.annotated_data_fixed_train_path

input_dir_path_testing =config.original_data_fixed_test_path
output_dir_path_testing = config.annotated_data_fixed_test_path
color_map = {
    "Mitosis":  (0, 0, 255),  # Red for Mitosis
    "Non-mitosis" : (0, 255, 0)   # Green for Non-mitosis
}
# Annotate training data
annotate_and_save_images("/home/aelkhate/anaconda3/envs/glioma-prep/Glioma-mdc2025/data/preprocessed_data_3/train", "/home/aelkhate/anaconda3/envs/glioma-prep/Glioma-mdc2025/data/preprocessed_data/train", color_map)
# Annotate test data
annotate_and_save_images("/home/aelkhate/anaconda3/envs/glioma-prep/Glioma-mdc2025/data/preprocessed_data_3/test","/home/aelkhate/anaconda3/envs/glioma-prep/Glioma-mdc2025/data/preprocessed_data/test", color_map)



In [ ]:
# Copy the json files to the annotated_data folder, because only the images were copied in the previous step
glioma_utils.copy_json_files(config.annotated_data_fixed_train_path, "/home/aelkhate/anaconda3/envs/glioma-prep/Glioma-mdc2025/data/preprocessed_data_1/train")
glioma_utils.copy_json_files(config.annotated_data_fixed_test_path,"/home/aelkhate/anaconda3/envs/glioma-prep/Glioma-mdc2025/data/preprocessed_data_1/test")
